In [ ]:
#| default_exp paramviewer

In [ ]:
from fairdatanow import RemoteData2, DataViewer
import panel as pn
import os

In [ ]:
@pn.cache()
def cache_data(df):
    return df

In [ ]:
configuration = {
    'url': "https://laboppad.nl/falnama-project", 
    'user':    os.getenv('NC_AUTH_USER'),
    'password': os.getenv('NC_AUTH_PASS')
}

In [ ]:
rmd = RemoteData2(configuration)
file_table = rmd.listdir("falnama-project")
df = cache_data(rmd.df)

Please wait while scanning all file paths in remote folder...


In [ ]:
DataViewer(data=df)

DataViewer(columns=['path'], data=                                                   path       size  \
2     falnama-project/.attachments.12065/Falnama_por...  749.2 KiB   
4        falnama-project/.jupyter/desktop-settings.json   32 Bytes   
6     falnama-project/.jupyter/desktop-workspaces/de...    1.7 KiB   
8     falnama-project/01. KEY INFO Project/1991-6-18...   26.6 MiB   
9     falnama-project/01. KEY INFO Project/1996-2-20...   30.7 MiB   
...                                                 ...        ...   
6383  falnama-project/OPENDATA/maxrf/previews/WM-718...  475.9 KiB   
6384                falnama-project/OPENDATA/xrays.html    2.7 MiB   
6385                          falnama-project/README.md  642 Bytes   
6387     falnama-project/Uploadfolder/firstdownload.txt   22 Bytes   
6388  falnama-project/filetree-laboppad-falnama_2025...  496.2 KiB   

                        ext  byte_size                  modified  isdir  
2                      .jpg     767156 2024-10-11 08:28:52+00:00  False  
4                     .json         32 2024-10-18 12:22:13+00:00  False  
6     .jupyterlab-workspace       1702 2024-10-18 12:22:04+00:00  False  
8                      .pdf   27843171 2022-10-07 09:41:35+00:00  False  
9                      .pdf   32179178 2022-09-20 08:47:34+00:00  False  
...                     ...        ...                       ...    ...  
6383                   .png     487327 2025-02-21 13:08:04+00:00  False  
6384                  .html    2796717 2025-08-01 11:19:27+00:00  False  
6385                    .md        642 2025-02-05 10:02:18+00:00  False  
6387                   .txt         22 2025-07-22 09:10:09+00:00  False  
6388                   .txt     508080 2025-05-08 08:20:33+00:00  False  

[5834 rows x 6 columns], filtered_data=                                                   path
262   falnama-project/05. Photos Objects/01. Photos ...
263   falnama-project/05. Photos Objects/01. Photos ...
1431  falnama-project/07. Results/01. Xray-images (R...
1432  falnama-project/07. Results/01. Xray-images (R...
1433  falnama-project/07. Results/01. Xray-images (R...
...                                                 ...
6008  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...
6009  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...
6010  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...
6011  falnama-project/OPENDATA/RMA_xrays/File_inform...
6384                falnama-project/OPENDATA/xrays.html

[74 rows x 1 columns], name='DataViewer00130', search='xray')

In [ ]:
#| export 

import panel as pn
import param
from panel.viewable import Viewer
import pandas as pd

pn.extension("tabulator")

In [ ]:
#| export 

class DataViewer(Viewer):
    data = param.DataFrame()

    columns = param.ListSelector(default=["path"])

    search = param.String(default="xray")
    
    filtered_data = param.DataFrame()

    def __init__(self, **params):
        super().__init__(**params)
        self.param.columns.objects = self.data.columns.to_list()

    @param.depends("data", "columns", "search", watch=True, on_init=True)
    def _update_filtered_data(self):
        df = self.data
        searched_df = df[df["path"].str.contains(self.search)]
        self.filtered_data = searched_df[self.columns]
    
    @param.depends("filtered_data")
    def number_of_rows(self):
        return f"Rows: {len(self.filtered_data)}"
    
    def __panel__(self):
        return pn.Column(
            pn.Row(pn.widgets.TextInput.from_param(self.param.search), pn.widgets.MultiChoice.from_param(self.param.columns)),
            pn.widgets.Tabulator(self.param.filtered_data, height=350, pagination=None, show_index=False, selectable=True, disabled=True),
            self.number_of_rows
        )